# tracktop lib Usage

In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append('../utils')
from tracktop import *

## Open dataframes from csv

In [2]:
# Open the input file as a DF
tracks      = pd.read_csv('/data/track-ml/output/test_real.csv',header = None)

# removing 6 fisrt columns information and de last column (real or fake)
tracks_real = tracks.iloc[:,7:-1]

tracks      = pd.read_csv('/data/track-ml/output/trtomei_fake.csv',header = None)

# removing 6 fisrt columns information and de last column (real or fake)
tracks_fake = tracks.iloc[:,7:-1]

# Replacing 0.0 -> infinite representation
# to move the hits with zeroes to the bottom of the dataframe
tracks_real_sort = tracks_real.replace(0.0, np.PINF).copy()
tracks_fake_sort = tracks_fake.replace(0.0, np.PINF).copy()

print('shape of tracks_real: ' + str(tracks_real.shape))
print('shape of tracks_fake: ' + str(tracks_fake.shape))

shape of tracks_real: (9367, 114)
shape of tracks_fake: (10000, 114)


In [4]:
def rho_from_xy(x, y):
    return np.sqrt(x * x + y * y)


def eta_from_theta(theta):
    return -np.log(np.tan(theta / 2.0))


def eta_from_xyz(x, y, z):
    theta = np.arctan2(np.sqrt(x * x + y * y), z)
    return eta_from_theta(theta)


def phi_from_xy(x, y):
    return np.arctan2(y, x)


def convert_xyz_to_rhoetaphi(x, y, z):
    return rho_from_xy(x, y), eta_from_xyz(x, y, z), phi_from_xy(x, y)


def convert_rhoetaphi_to_xyz(rho, eta, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    z = rho * np.sinh(eta)
    return x, y, z


In [10]:
x = 10
y = 20
z = 30

rho, eta, phi = convert_xyz_to_rhoetaphi(x, y, z)


ValueError: not enough values to unpack (expected 4, got 3)

## Sorting the dataframe using xyz_bsort function

### REAL tracks

In [ ]:
#for i in range(int(tracks_real_sort.shape[0])):
for i in range(10):
    xyz_bsort(tracks_real_sort.iloc[i,:])

# Replacing infinite representation -> 0.0    
tracks_real_sort = tracks_real_sort.replace(np.PINF,0.0).copy()

### FAKE tracks

In [ ]:
#for i in range(int(tracks_fake_sort.shape[0])):
for i in range(10):
    xyz_bsort(tracks_fake_sort.iloc[i,:])

    # Replacing infinite representation -> 0.0   
tracks_fake_sort = tracks_fake_sort.replace(np.PINF,0.0).copy()

### Ploting dataframe of tracks

**Default data:** A dataframe with one track perline and just 6 hit information 

In [ ]:
track_plot(tracks_real_sort)

### Ploting using parameters

In [ ]:
track_plot(tracks_fake_sort, track_color = 'blue', n_tracks = 10, title = 'Fake tracks sorted')